# Máquinas Vectoriales de Soporte (SVM)

### 2. Transformación de los datos para SVM

In [ ]:
# Importar librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Cargar datos procesados
df = pd.read_csv("../data/train_preprocessed.csv")

# Crear la variable categórica de precios
def categorizar_precio(valor):
    if valor < 150000:
        return 'barata'
    elif valor < 300000:
        return 'media'
    else:
        return 'cara'

df['PrecioCategoria'] = df['SalePrice'].apply(categorizar_precio)

# Eliminar SalePrice para clasificación
df_clasificacion = df.drop(columns=['SalePrice'])

# Separar variables predictoras (X) y target (y)
X = df_clasificacion.drop(columns=['PrecioCategoria'])
y = df_clasificacion['PrecioCategoria']

# Codificar variables categóricas si las hay
X = pd.get_dummies(X)

# Estandarizar características numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Codificar la variable objetivo (barata, media, cara → 0, 1, 2)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Dividir en entrenamiento y prueba 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)
